In [1]:
!pip install spacy==3.0.0
!pip install sklearn_crfsuite
!pip install scikit-learn==0.23.2
!pip install pytorch-crf

     |████████████████████████████████| 12.7 MB 5.4 MB/s 
     |████████████████████████████████| 628 kB 53.6 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 9.1 MB 26.1 MB/s 
     |████████████████████████████████| 451 kB 66.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 743 kB 5.0 MB/s 
     |████████████████████████████████| 6.8 MB 

In [2]:
! wget https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio

--2022-02-22 08:49:19--  https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 759757 (742K)
Saving to: ‘restauranttrain.bio’

restauranttrain.bio 100%[===================>] 741.95K  --.-KB/s    in 0.1s    

2022-02-22 08:49:19 (6.35 MB/s) - ‘restauranttrain.bio’ saved [759757/759757]



In [3]:
# import main libraries
from IPython.display import Image
from IPython.core.display import display, HTML
import pandas as pd
from collections import Counter
import random
from spacy import displacy
import json
import os
from spacy.training import offsets_to_biluo_tags
import spacy
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

In [ ]:
with open('restauranttrain.bio', 'r') as f:
    for line in f.readlines()[:30]:
        print(line)

In [7]:
# reformatting the data
with open('restauranttrain.bio', 'r') as f:
    with open('restauranttrain_updated.bio', 'w') as w:
        for line in f.readlines():
            if line == '\n':
                w.write(line)
            else:
                w.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')

In [20]:
data = []
labels = []
with open('restauranttrain_updated.bio', 'r') as f:
    cur_data = []
    cur_label = []
    for line in f.readlines():
        if line == '\n':
            data.append(cur_data)
            labels.append(cur_label)
            cur_data = []
            cur_label = []
        else:
            cur_data.append(line.strip().split('\t')[0])
            cur_label.append(line.strip().split('\t')[1])

In [21]:
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

2	B-Rating
start	I-Rating
restaurants	O
with	O
inside	B-Amenity
dining	I-Amenity


In [22]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(10)

,sent_id,data,entities
0,0,2,B-Rating
1,0,start,I-Rating
2,0,restaurants,O
3,0,with,O
4,0,inside,B-Amenity
5,0,dining,I-Amenity
6,1,34,O
7,2,5,B-Rating
8,2,star,I-Rating
9,2,resturants,O


In [23]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)
sentences = getter.sentences

In [24]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [25]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [26]:
X_train = X[:7000]
X_test = X[7000:]
y_train = y[:7000]
y_test = y[7000:]

In [27]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 7000/7000 [00:00<00:00, 12345.89it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 25163
Seconds required: 0.110

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.24  loss=137545.30 active=24772 feature_norm=1.00
Iter 2   time=0.39  loss=114370.01 active=24415 feature_norm=3.03
Iter 3   time=0.13  loss=99647.40 active=23572 feature_norm=2.33
Iter 4   time=0.12  loss=97450.70 active=24563 feature_norm=2.44
Iter 5   time=0.12  loss=83067.15 active=23868 feature_norm=11.17
Iter 6   time=0.13  loss=57827.41 active=24641 feature_norm=10.31
Iter 7   time=0.12  loss=55162.38 active=24682 feature_norm=10.96
Iter 8   time=0.12  loss=46035.36 active=24556 feature_norm=15.19
Iter 9   time=0.13  loss=39461.19 active=24562 feature_norm=17.96
Iter 10

In [28]:
all_entities = sorted(df.entities.unique().tolist())

In [29]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.900438090391091

In [30]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=[i for i in all_entities if i != 'O'])

0.8281450602222045

In [31]:
print(metrics.flat_classification_report(y_test, y_pred, labels = all_entities))

                   precision    recall  f1-score   support

        B-Amenity       0.79      0.72      0.75       188
        B-Cuisine       0.83      0.89      0.86       243
           B-Dish       0.78      0.80      0.79       137
          B-Hours       0.84      0.78      0.81        63
       B-Location       0.93      0.90      0.91       507
          B-Price       0.90      0.87      0.89        62
         B-Rating       0.87      0.90      0.89       143
B-Restaurant_Name       0.88      0.81      0.84       139
        I-Amenity       0.78      0.71      0.74       191
        I-Cuisine       0.72      0.77      0.74        66
           I-Dish       0.76      0.75      0.76        89
          I-Hours       0.86      0.95      0.90        80
       I-Location       0.82      0.78      0.80       207
          I-Price       0.67      0.43      0.52        14
         I-Rating       0.74      0.73      0.73        55
I-Restaurant_Name       0.75      0.79      0.77       

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-Amenity', 'B-Cuisine', 'B-Dish', 'B-Hours', 'B-Location', 'B-Price', 'B-Rating', 'B-Restaurant_Name', 'I-Amenity', 'I-Cuisine', 'I-Dish', 'I-Hours', 'I-Location', 'I-Price', 'I-Rating', 'I-Restaurant_Name', 'O'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
